In [1]:
# A description of the data and how it will be used to solve the problem. (15 marks)


##  2. Data Section:   
### Description of the data and its sources that will be used to solve the problem  
### Description of the Data:  
The following data is required to answer the issues of the problem:  
  
   * List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)  
    * List of Subway metro stations in Manhattan with their address location  
    * List of apartments for rent in Manhattan area with their addresses and price  
    * Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc  
    * Venues for each Manhattan neighborhood ( than can be clustered)  
    * Venues for subway metro stations, as needed  
  
### How the data will be used to solve the problem  
  
The data will be used as follows:  
  
   * Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)  
    * Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately  
    * Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.  
    * create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.  
    * Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.  
    * Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.  
  
The procesing of these DATA will allow to answer the key questions to make a decision:  
  
   * what is the cost of rent (per square ft) around a mile radius from each subway metro station?  
    * what is the area of Manhattan with best rental pricing that meets criteria established?  
    * What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?  
    * What are the venues of the two best places to live? How the prices compare?  
    * How venues distribute among Manhattan neighborhoods and around metro stations?  
    * Are there tradeoffs between size and price and location?  
    * Any other interesting statistical data findings of the real estate and overall data.  
    * Reference of venues around current residence in Singapore for comparison to Manhattan place  
  
  
### Reference of venues around current residence in Singapore for comparison to Manhattan place

In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.50 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  39.00 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  42.02 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  50.63 MB/s
Libraries imported.


In [3]:
# Shenton Way, District 01, Singapore
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore home are 1.2787941, 103.848846.


In [4]:
neighborhood_latitude=1.2792655
neighborhood_longitude=103.8480938

In [7]:
CLIENT_ID = 'G1LIJ4IQ3IPJK20G0TRZIMC145SZWKFKCPCAKEVTEYQ1MMAZ'
CLIENT_SECRET = 'YS5NPIDD1RQHRA1IIAH121SWBTO1RQZPJXXJVPVHJTLANQJR'
VERSION = '20190421'

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=G1LIJ4IQ3IPJK20G0TRZIMC145SZWKFKCPCAKEVTEYQ1MMAZ&client_secret=YS5NPIDD1RQHRA1IIAH121SWBTO1RQZPJXXJVPVHJTLANQJR&v=20190421&ll=1.2792655,103.8480938&radius=500&limit=100'

In [9]:
results = requests.get(url).json()

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Freehouse,Beer Garden,1.281254,103.848513
3,Park Bench Deli,Deli / Bodega,1.279872,103.847287
4,Matt's | The Chocolate Shop,Dessert Shop,1.280462,103.846950
5,PS.Cafe,Café,1.280468,103.846264
6,Amoy Street Food Centre,Food Court,1.279368,103.847079
7,왕대박 Wang Dae Bak Korean BBQ Restaurant,Korean Restaurant,1.281345,103.847551
8,Magal BBQ 마포갈매기,Korean Restaurant,1.281299,103.847932
9,Mellower Coffee,Café,1.277814,103.848188


In [12]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg